In [1]:
import googlemaps
from dataclasses import dataclass, fields
import os
import datetime
import numpy as np
import pandas as pd

from enum import Enum, auto
from io import StringIO
import typing as t
from functools import cached_property

In [12]:
gmaps = googlemaps.Client(key=os.getenv("GMAPS_KEY"))

In [3]:
def address_to_lat_long(address: str) -> tuple[float, float]:
    """Uses the google maps API to convert an address to a lat/long pair."""
    place = gmaps.geocode(address)
    geometry = place[0]["geometry"]

    if "location" in geometry:
        loc = geometry["location"]
        return loc["lat"], loc["lng"]
    elif "bounds" in geometry:
        bounds = geometry["bounds"].values()
        lat = np.mean(k["lat"] for k in bounds)
        long = np.mean(k["long"] for k in bounds)

        return lat, long
    else:
        raise

In [4]:
def transit_cost(
    start: str, 
    end: str,
    time: datetime.datetime = datetime.datetime.strptime("2024-4-01 10:00", "%Y-%m-%d %H:%M"),
    hourly_rate = 100.0,
) -> float:
    """Computes the transit cost between two destinations including fare and time cost based on an hourly rate."""
    directions = gmaps.directions(start, end, mode="transit", departure_time=time)[0]

    if "fare"  in directions:        
        transit_cost = directions["fare"]["value"]
    else:
        transit_cost = 0
    
    legs = directions["legs"][0]

    if "arrival_time" in legs:
        arriv_unix = legs["arrival_time"]["value"]
        depart_unix = legs["departure_time"]["value"]
        transit_time_secs = arriv_unix - depart_unix
    elif "duration" in legs:
        transit_time_secs = legs["duration"]["value"]
    else:
        raise ValueError(legs)

    hourly_cost = transit_time_secs / 3600.0 * hourly_rate

    return hourly_cost + transit_cost

In [5]:
def find_nearest(start: str, query: str, limit_to: int = 3) -> list[t.Any]:
    """Finds the `limit_to` nearest results for `query` relative to `start`. Return a list of dicts."""
    results = gmaps.places(query=query, location=address_to_lat_long(start))["results"]
    relevant_results = [res for res in results if res["business_status"] == "OPERATIONAL"][:limit_to]
    return relevant_results

In [6]:
def oneway_cost_to_nearest_places(start: str, places: list[t.Any]) -> float:
    """Finds the cheapest one-way transit cost from `start` to anything in `places`."""
    return np.min([transit_cost(start, result["formatted_address"]) for result in places])

In [7]:
def oneway_cost_to_nearest(start: str, query: str) -> float:
    """Finds the cheapest one-way transit cost from `start` to some place described by `query`."""
    places = find_nearest(start, query)
    return oneway_cost_to_nearest_places(start, places)

In [8]:
def oneway_cost_to_campus(start: str) -> float:
    """Finds the one-way transit cost from `start` to BUMC."""
    BUMC = "72 E Concord St, Boston, MA 02118"
    return oneway_cost_to_nearest_places(start, [gmaps.geocode(BUMC)[0]])

In [9]:
BOSTON_GYM_MONTHLY_COST = 50.0
BOSTON_LAUNDROMAT_PER_LOAD_COST = 6.0

N_LOADS_LAUNDRY_PER_WEEK = 2.5
N_TIMES_ON_CAMPUS_PER_WEEK = 5
N_TIMES_GYM_PER_WEEK = 5

N_WEEKS_PER_MONTH = 4.35

class Laundry(Enum):
    IN_UNIT = "IN_UNIT"
    IN_BUILDING = "IN_BUILDING"
    OTHER = "OTHER"

@dataclass
class Apartment:
    # Metadata
    name: str
    address: str
    neighborhood: str

    # Quantitative detail
    monthly_rent: float
    size: int

    # Features
    has_gym: bool
    laundry: Laundry

    def __post_init__(self):
        for field in fields(self):
            value = getattr(self, field.name)
            if not isinstance(value, field.type):
                setattr(self, field.name, field.type(value))


    @cached_property
    def nearby_gyms(self) -> list[t.Any]:
        return find_nearest(self.address, "gym")


    @cached_property
    def gym_monthly_cost(self) -> float:
        base = 0.0
        if not self.has_gym:
            base += BOSTON_GYM_MONTHLY_COST
            base += (
                (2 * oneway_cost_to_nearest_places(self.address, self.nearby_gyms)) 
                * N_TIMES_GYM_PER_WEEK 
                * N_WEEKS_PER_MONTH
            )
        return base

    @cached_property
    def bu_commute_cost(self) -> float:
        return (
            (2.0 * oneway_cost_to_campus(self.address))
            * N_TIMES_ON_CAMPUS_PER_WEEK
            * N_WEEKS_PER_MONTH
        )

    @cached_property
    def real_monthly_cost(self) -> float:
        base = self.monthly_rent
        base += self.gym_monthly_cost
        base += self.bu_commute_cost
        return base

In [10]:
csv = """
name,address,monthly_rent,has_gym,laundry,size,neighborhood
1550 on the charles,"1550 Soldiers Field Rd, Boston, MA 02135","$2,728",TRUE,IN_UNIT,632,cambridge
park 151,"151 N 1st St., Boston, MA","$3,595",TRUE,IN_UNIT,579,allston
burbank apts,"18 Haviland St, Boston, MA 02115",,,IN_UNIT,385,fenway
columbia flats,"1258 Massachusetts Ave, Dorchester, MA 02125","$3,000",TRUE,IN_UNIT,108,dorchester
piano craft guild,"791 Tremont St, Boston, MA 02118","$3,600",TRUE,IN_BUILDING,1250,south end
the andi,"4 Lucy St, Boston, MA 02125","$3,136",TRUE,IN_UNIT,754,dorchester
125 warren,"125 Warren St, Boston, MA 02119","$2,900",FALSE,IN_BUILDING,830,lower roxbury
the melnea residences,"431 Melnea Cass Blvd, Boston, MA 02119","$3,590",FALSE,IN_UNIT,907,south end/roxbury
the cara,"13 Shetland St, Roxbury, MA 02119","$2,850",TRUE,IN_UNIT,706,roxbury
dot block,"1211 Dorchester Ave, Boston, MA 02125","$2,900",TRUE,IN_UNIT,551,dorchester
metromark,"3593-3615 Washington St, Jamaica Plain, MA 02130","$2,614",TRUE,IN_UNIT,676,jamaica plain
flats on d,"411 D St, Boston, MA 02210","$3,400",TRUE,IN_UNIT,730,waterfront
hub25,"25 Morrissey Blvd, Boston, MA 02125","$2,952",TRUE,IN_UNIT,680,columbia point
3200 washington,"3200 Washington St, Boston, MA 02130","$3,200",TRUE,IN_UNIT,650,jamaica plain
250 centre,"250 Centre St, Boston, MA 02119","$2,795",TRUE,IN_UNIT,635,jamaica plain
the laneway,"9 Burney St, Boston, MA 02120","$3,200",TRUE,IN_UNIT,577,mission hill
carson towers,"1410 Columbia Rd, Boston, MA 02127","$2,850",TRUE,IN_BUILDING,557,waterfront
the monroe,"105 Washington St, Brighton, MA 02135","$3,500",TRUE,IN_UNIT,550,brighton
south standard,"235 Old Colony Ave, Boston, MA 02127","$3,345",TRUE,IN_UNIT,688,waterfront/southside"""
df = pd.read_csv(StringIO(csv)).dropna()
df["monthly_rent"] = df["monthly_rent"].str.replace(r"[^.0-9\-]", "", regex=True)
apts = df.apply(lambda r: Apartment(**r), axis=1)
apts[0]

Apartment(name='1550 on the charles', address='1550 Soldiers Field Rd, Boston, MA 02135', neighborhood='cambridge', monthly_rent=2728.0, size=632, has_gym=True, laundry=<Laundry.IN_UNIT: 'IN_UNIT'>)

In [11]:
[(x, x.real_monthly_cost) for x in sorted(apts, key=lambda x: x.real_monthly_cost)]

[(Apartment(name='the cara', address='13 Shetland St, Roxbury, MA 02119', neighborhood='roxbury', monthly_rent=2850.0, size=706, has_gym=True, laundry=<Laundry.IN_UNIT: 'IN_UNIT'>),
  3824.1583333333333),
 (Apartment(name='columbia flats', address='1258 Massachusetts Ave, Dorchester, MA 02125', neighborhood='dorchester', monthly_rent=3000.0, size=108, has_gym=True, laundry=<Laundry.IN_UNIT: 'IN_UNIT'>),
  4209.783333333333),
 (Apartment(name='the andi', address='4 Lucy St, Boston, MA 02125', neighborhood='dorchester', monthly_rent=3136.0, size=754, has_gym=True, laundry=<Laundry.IN_UNIT: 'IN_UNIT'>),
  4325.241666666667),
 (Apartment(name='piano craft guild', address='791 Tremont St, Boston, MA 02118', neighborhood='south end', monthly_rent=3600.0, size=1250, has_gym=True, laundry=<Laundry.IN_BUILDING: 'IN_BUILDING'>),
  4426.741666666667),
 (Apartment(name='south standard', address='235 Old Colony Ave, Boston, MA 02127', neighborhood='waterfront/southside', monthly_rent=3345.0, size=6